# Important Resources:

<a href="https://pennylane.ai/qml/demos/tutorial_quantum_chemistry/"> Building Molecular Hamiltonians </a>

<a href="https://pennylane.ai/qml/demos/tutorial_vqe/"> A brief overview of VQE </a>

<a href="https://www.youtube.com/watch?v=4Xnxa6tzPeA"> Variational Quantum Eigensolver </a>

<a href="https://docs.pennylane.ai/en/stable/introduction/chemistry.html"> Quantumm Chemistry Documentation </a>

In [1]:
import pennylane as qml
import pennylane.numpy as np

In [3]:
def hydrogen_hamiltonian(coordinates, charge):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.
    
    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(
        ["H", "H"], coordinates, charge, basis="STO-3G"
    )[0]

def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.
    
    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns: 
        (int): The number of electrons.
    """
        
    return 2 - charge

In [4]:
def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.
    
    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
        # Put your solution here #
    return qml.qchem.hf_state(electrons, num_qubits)

In [5]:
def run_VQE(coordinates, charge):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """

    hamiltonian = hydrogen_hamiltonian(np.array(coordinates), charge)

    electrons = num_electrons(charge)
    num_qubits = len(hamiltonian.wires)

    hf_state = hf(electrons, num_qubits)
    # singles and doubles are used to make the AllSinglesDoubles template
    singles, doubles = qml.qchem.excitations(electrons, num_qubits)

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
        """
                # Put your solution here #
        qml.templates.AllSinglesDoubles(weights, dev.wires, hf_state, 
                                        singles, doubles)
        return qml.expval(hamiltonian)
        np.random.seed = 1234
    weights = np.random.normal(
        0, np.pi, len(singles) + len(doubles), requires_grad=True
    )
    opt = qml.AdamOptimizer(0.5)

    for _ in range(200):
        weights = opt.step(cost, weights)

    return cost(weights)

In [7]:
print(run_VQE([0.0, 0.0, -0.8, 0.0, 0.0, 0.8], -1))

-0.5316836600593313
